# Project Overview:
This notebook implements a Vision Transformer (ViT) model for multi-class skin cancer classification using dermatoscopic images. The goal is to train a deep learning model capable of accurately classifying skin lesions into predefined diagnostic categories.

The workflow includes:

-Dataset loading and preprocessing

-Data augmentation

-Model initialization using a pretrained ViT

-Training and validation

-Model evaluation and testing

In [ ]:
!pip install -q torch torchvision timm scikit-learn albumentations pandas matplotlib tqdm


In [ ]:
from google.colab import files
files.upload()   # ⬅️ choose your kaggle.json


In [ ]:
!mv "kaggle (17).json" kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list -s ham10000


# Connect to HAM10000 dataset

In [ ]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p /content --unzip


In [ ]:
!mkdir -p /content/HAM10000_images
!cp /content/HAM10000_images_part_1/* /content/HAM10000_images/
!cp /content/HAM10000_images_part_2/* /content/HAM10000_images/


Import necessary libraries

In [ ]:
import torch, timm, numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import pandas as pd, os

# ==== CONFIG ====
CSV_FILE = "/content/HAM10000_metadata.csv"
IMG_DIR = "/content/HAM10000_images"
OUT_DIR = "/content/outputs"
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LR = 3e-5
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUT_DIR, exist_ok=True)
print("Using device:", device)

# ==== LOAD DATA ====
df = pd.read_csv(CSV_FILE)


Set Directory Path

In [ ]:
CSV_FILE = "/content/HAM10000_metadata.csv"
IMG_DIR = "/content/HAM10000_images"
OUT_DIR = "/content/outputs"

import os
os.makedirs(OUT_DIR, exist_ok=True)
print("✅ Paths ready")


In [ ]:
import torch, timm, numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np


In [ ]:
df = pd.read_csv(CSV_FILE)
print("✅ Metadata shape:", df.shape)
print("✅ Sample columns:", df.columns.tolist())

import os
print("✅ Total images:", len(os.listdir(IMG_DIR)))
df.head()


# Dataset and Preprocessing
The dataset consists of labeled dermatoscopic images organized into class-specific directories. Images are resized and normalized to match the input requirements of the Vision Transformer.

*Image Parameters*

-Image size: 224 × 224

-Color channels: 3 (RGB)

*Data Augmentation (Training Only)*

-Random resized cropping

-Horizontal flipping

-Vertical flipping

-Brightness and contrast adjustment

-Normalization

Validation and test images are only resized and normalized to ensure unbiased evaluation.

In [ ]:
# ==== CONFIG ====
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
LR = 3e-5
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# ==== PREPARE DATA ====
df['label_str'] = df['dx']
classes = sorted(df['label_str'].unique())
df['label'] = df['label_str'].map({c:i for i,c in enumerate(classes)})

train_df, test_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['label'], random_state=42)

#remove hair_removal
def remove_hair(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #black-hat filtering
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    #threshold
    _, thresh = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)

    #inpaint
    image=cv2.inpaint(image,thresh,1,cv2.INPAINT_TELEA)
    return image

def get_transforms(phase):
    if phase == 'train':
        return A.Compose([
            A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.8, 1.0)),
            A.HorizontalFlip(),
            A.VerticalFlip(p=0.2),
            A.RandomBrightnessContrast(p=0.5),

            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(height=IMG_SIZE, width=IMG_SIZE),
            A.Normalize(),
            ToTensorV2(),
        ])


class SkinDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(os.path.join(self.img_dir, row['image_id'] + '.jpg')).convert('RGB'))
        if self.transform: img = self.transform(image=img)['image']
        label = int(row['label'])
        return img, label

train_loader = DataLoader(SkinDataset(train_df, IMG_DIR, get_transforms('train')), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(SkinDataset(val_df, IMG_DIR, get_transforms('val')), batch_size=BATCH_SIZE)

# ==== MODEL ====
# Adding drop_out rate to the model
model = timm.create_model("vit_base_patch16_224", pretrained=True, num_classes=len(classes),drop_rate=0.3,drop_path_rate=0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-2)

# ==== TRAINING ====
best_acc = 0
train_losses, val_accuracies, train_accs = [], [], []  # added train_accs list

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # === Compute Training Accuracy ===
    model.eval()
    train_preds, train_true = [], []
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            train_preds.extend(outputs.argmax(1).cpu().numpy())
            train_true.extend(labels.cpu().numpy())
    train_acc = accuracy_score(train_true, train_preds)

    # === Validation ===
    preds, true = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds.extend(outputs.argmax(1).cpu().numpy())
            true.extend(labels.cpu().numpy())
    val_acc = accuracy_score(true, preds)

    # === Log and Save ===
    print(f"Epoch {epoch+1} | Train Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    train_losses.append(running_loss / len(train_loader))
    val_accuracies.append(val_acc)
    train_accs.append(train_acc)

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(OUT_DIR, "best_model.pth"))
        print("✅ Saved best model")


# === Save Final Model ===
torch.save(model.state_dict(), os.path.join(OUT_DIR, "final_model.pth"))
print("✅ Final model saved manually!")
print("Training Complete! Best Val Accuracy:", best_acc)

Visualize some mis-classified images

In [ ]:
import matplotlib.pyplot as plt

# ==== LOAD BEST MODEL ====
model.load_state_dict(torch.load(os.path.join(OUT_DIR, "best_model.pth"), map_location=device))
model.eval()

# ==== PREDICT ON VALIDATION SET ====
misclassified = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        for i in range(len(preds)):
            if preds[i] != labels[i]:
                misclassified.append((images[i].cpu(), preds[i].cpu().item(), labels[i].cpu().item()))

# ==== SHOW SOME MISCLASSIFIED IMAGES ====
def imshow(img, title):
    img = img.permute(1, 2, 0).numpy()
    img = (img - img.min()) / (img.max() - img.min())  # normalize for display
    plt.imshow(img)
    plt.title(title)
    plt.axis('off')

# Display first 6 misclassified samples
plt.figure(figsize=(12, 8))
for i, (img, pred, true) in enumerate(misclassified[:6]):
    plt.subplot(2, 3, i + 1)
    imshow(img, f"Pred: {classes[pred]}\nTrue: {classes[true]}")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(range(1, EPOCHS+1), train_losses, marker='o')
plt.title("Training Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.subplot(1,2,2)
plt.plot(range(1, EPOCHS+1), val_accuracies, marker='o', color='orange')
plt.title("Validation Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()


# Implement Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(xticks_rotation=45, cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# Report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=classes))


# Model Evaluation

In [ ]:
# === Validation Curve (Training vs Validation Accuracy) ===
import matplotlib.pyplot as plt

epochs = range(1, len(val_accuracies) + 1)

plt.figure(figsize=(7,5))
plt.plot(epochs, train_accs, label='Train Accuracy', color='royalblue')
plt.plot(epochs, val_accuracies, label='Validation Accuracy', color='darkorange')

plt.title('Training vs Validation Accuracy ViT Model', fontsize=13, weight='bold')
plt.xlabel('Epoch', fontsize=11)
plt.ylabel('Accuracy', fontsize=11)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
model = timm.create_model("vit_base_patch16_224", pretrained=False, num_classes=len(classes)).to(device)
model.load_state_dict(torch.load("/content/outputs/best_model.pth", map_location=device))
model.eval()
print("✅ Model loaded successfully for evaluation.")


In [ ]:
# ==== EVALUATION FOR TRAIN, VAL, AND TEST SETS ====
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# === Load the best model ===
best_model_path = os.path.join(OUT_DIR, "best_model.pth")
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    print(f"✅ Loaded best model from {best_model_path}")
else:
    print("⚠️ Best model not found, using current weights")

# === Helper Function for Confusion Matrix & Report ===
def evaluate_model(loader, split_name="Validation"):
    y_true, y_pred = [], []
    model.eval()
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    # === Accuracy ===
    acc = accuracy_score(y_true, y_pred)
    print(f"\n📊 {split_name} Accuracy: {acc:.4f}")

    # === Confusion Matrix ===
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"{split_name} Confusion Matrix ViT Model")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # === Normalized Confusion Matrix ===
    plt.figure(figsize=(8, 6))
    cm_norm = confusion_matrix(y_true, y_pred, normalize='true')
    sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap="Greens",
                xticklabels=classes, yticklabels=classes)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"{split_name} Normalized Confusion Matrix ViT Model")
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # === Classification Report ===
    print(f"\n🧾 {split_name} Classification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))

# === Evaluate on all splits ===
evaluate_model(train_loader, "Training")
evaluate_model(val_loader, "Validation")
evaluate_model(DataLoader(SkinDataset(test_df, IMG_DIR, get_transforms('val')), batch_size=BATCH_SIZE), "Test")

In [ ]:
from torch import nn

# 1) Find the first Conv2d layer
first_conv_name, first_conv = None, None
for name, module in model.named_modules():
    if isinstance(module, nn.Conv2d):
        first_conv_name, first_conv = name, module
        break

print("First Conv layer:", first_conv_name, first_conv)

# 2) Register a forward hook to capture its output
feature_maps = {}

def hook_fn(module, input, output):
    feature_maps['first_conv'] = output.detach().cpu()

hook_handle = first_conv.register_forward_hook(hook_fn)


In [ ]:
!pip install torchinfo


# Visualizing model architecture details

In [ ]:
from torchinfo import summary

# For ViT base, input is 3×224×224; batch size 1
summary(
    model,
    input_size=(1, 3, 224, 224),
    col_names=("input_size", "output_size", "num_params", "kernel_size"),
    depth=4,   # increase if you want to expand deeper nested blocks
)


# Visualize simplified ViT architecture

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

# ===== Helper: draw a fake 3D block =====
def draw_block(ax, x, y, w, h, d, label, face_color="#DDDDDD"):
    """
    x, y : front-bottom-left corner
    w, h : width, height of front face
    d    : depth offset
    """
    # Front rectangle
    front = [(x, y), (x+w, y), (x+w, y+h), (x, y+h)]
    # Top rectangle
    top = [(x, y+h), (x+d, y+h+d), (x+w+d, y+h+d), (x+w, y+h)]
    # Side rectangle
    side = [(x+w, y), (x+w+d, y+d), (x+w+d, y+h+d), (x+w, y+h)]

    for poly_pts in [front, top, side]:
        poly = Polygon(poly_pts, closed=True,
                       facecolor=face_color, edgecolor="black")
        ax.add_patch(poly)

    # Put text on the front face
    ax.text(
        x + w/2,
        y + h/2,
        label,
        ha="center", va="center",
        fontsize=9
    )

# ===== Create the diagram =====
fig, ax = plt.subplots(figsize=(14, 4))

# Coordinates & sizes
y = 0
h = 2.8
w = 2.4
d = 0.5
gap = 1.2

x0 = 0
x1 = x0 + w + gap
x2 = x1 + w + gap
x3 = x2 + w + gap
x4 = x3 + w + gap

# Block 1: Input
label_input = "Input\n224×224×3"
draw_block(ax, x0, y, w, h, d, label_input, face_color="#F2F2F2")

# Block 2: Patch Embedding
label_patch = (
    "Patch Embedding\n"
    "16×16 Conv\n"
    "196 tokens × 768"
)
draw_block(ax, x1, y, w, h, d, label_patch, face_color="#D0E4F7")

# Block 3: Transformer Encoder
label_encoder = (
    "Transformer Encoder\n"
    "12 blocks\n"
    "MHSA + MLP"
)
draw_block(ax, x2, y, w, h, d, label_encoder, face_color="#D9EAD3")

# Block 4: MLP Head
label_head = (
    "MLP Head\n"
    "768 → C"
)
draw_block(ax, x3, y, w, h, d, label_head, face_color="#FCE5CD")

# Block 5: Output
label_output = "Output\nPredicted Class"
draw_block(ax, x4, y, w, h, d, label_output, face_color="#F4CCCC")

# Arrows between blocks
def arrow_between(x_start, x_end, y_center):
    ax.annotate(
        "",
        xy=(x_end - 0.2, y_center),
        xytext=(x_start + w + 0.2, y_center),
        arrowprops=dict(arrowstyle="->", linewidth=1.5)
    )

y_center = y + h/2
arrow_between(x0, x1, y_center)
arrow_between(x1, x2, y_center)
arrow_between(x2, x3, y_center)
arrow_between(x3, x4, y_center)

# Title
ax.text(
    (x0 + x4 + w) / 2, y + h + 1.0,
    "Simplified Vision Transformer Architecture",
    ha="center", va="center",
    fontsize=14, fontweight="bold"
)

ax.set_xlim(-1, x4 + w + 1)
ax.set_ylim(-1, y + h + 2)
ax.axis("off")

# No tight_layout to avoid margin warnings
plt.show()
